# Lab 3 (Alternate): Azure AI Foundry Agents Integration

**Purpose:** Learn how to use Azure AI Foundry Agents for sophisticated chat completion and agent-based interactions. This lab demonstrates how to leverage Azure AI Foundry's agent capabilities for more advanced conversational AI scenarios beyond simple chat completions.

## Overview

In this alternate Lab 3, we'll:
- Connect to Azure AI Foundry using the Agents SDK
- Create and configure AI agents with specific capabilities
- Test agent-based interactions for complex tasks
- Compare agent performance with direct model calls
- Understand when to use agents vs direct model access in our hybrid architecture

## Azure AI Foundry Agents Benefits
- **Structured Conversations**: Built-in conversation management
- **Tool Integration**: Native support for function calling and tools
- **State Management**: Automatic handling of conversation state
- **Advanced Capabilities**: Multi-turn reasoning, planning, and execution

## Step 3.1: Environment Setup and Azure AI Foundry Configuration

In [ ]:
# Add parent directory to path for module imports
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

# Standard library imports
import time
import json
from typing import Dict, List, Optional
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Load configuration
load_dotenv()

print("✅ Environment setup complete")
print(f"Python version: {sys.version}")
print(f"Working directory: {os.getcwd()}")

In [ ]:
from foundry_local import FoundryLocalManager

# Initialize and optionally bootstrap with a model
manager = FoundryLocalManager(alias_or_model_id=None, bootstrap=True)

LOCAL_ENDPOINT = manager.service_uri
LOCAL_MODEL_NAME = os.environ["LOCAL_MODEL_NAME"]
AZURE_OPENAI_API_VERSION = os.environ["AZURE_OPENAI_API_VERSION"]

print(f"Local service: {LOCAL_ENDPOINT}")
print(f"Local endpoint: {manager.endpoint}")
print(f"Local model alias: {LOCAL_MODEL_NAME}")

In [ ]:
# Azure AI Foundry and Agents configuration
AZURE_AI_FOUNDRY_ENDPOINT = os.environ["AZURE_AI_FOUNDRY_ENDPOINT"]
AZURE_AI_FOUNDRY_PROJECT_ENDPOINT = os.environ["AZURE_AI_FOUNDRY_PROJECT_ENDPOINT"]

# Azure OpenAI configuration (direct)
AZURE_OPENAI_ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_KEY = os.environ["AZURE_OPENAI_KEY"]
AZURE_OPENAI_DEPLOYMENT = os.environ["AZURE_DEPLOYMENT_NAME"]
AZURE_OPENAI_API_VERSION = os.environ["AZURE_OPENAI_API_VERSION"]


print("🔧 Configuration Summary:")
print(f"   Azure AI Project: {'✅ Configured' if AZURE_AI_FOUNDRY_ENDPOINT else '❌ Missing'}")
print(f"   Azure Foundry Project Endpoint: {AZURE_AI_FOUNDRY_PROJECT_ENDPOINT}")
print(f"   Azure OpenAI Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"   Azure Deployment: {AZURE_OPENAI_DEPLOYMENT}")
print(f"   API Version: {AZURE_OPENAI_API_VERSION}")
print(f"   Local Endpoint: {LOCAL_ENDPOINT}")
print(f"   Local Model: {LOCAL_MODEL_NAME}")

# Verify required configuration
required_vars = [AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_KEY, AZURE_OPENAI_DEPLOYMENT]
if not all(required_vars):
    print("\n❌ Missing required Azure configuration. Please check your .env file.")
    print("Required variables: AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_KEY, AZURE_DEPLOYMENT_NAME")
else:
    print("\n✅ All required Azure configuration loaded")

## Step 3.2: Install and Import Azure AI Foundry Agents SDK

We'll use the Azure AI Foundry Agents SDK for creating and managing AI agents.

In [ ]:
# Import Azure AI Foundry Agents and related libraries
try:
    from azure.ai.projects import AIProjectClient
    from azure.ai.agents.models import (
        CodeInterpreterTool
    )
    from azure.identity import DefaultAzureCredential
    from azure.ai.inference import ChatCompletionsClient
    from azure.ai.inference.models import SystemMessage, UserMessage
    print("✅ Azure AI Foundry Agents SDK imported successfully")
    foundry_agents_available = True
except ImportError as e:
    print(f"⚠️ Azure AI Foundry Agents SDK not available: {e}")
    print("   Falling back to direct Azure OpenAI client")
    foundry_agents_available = False

# Also import direct Azure OpenAI client as fallback
from openai import AzureOpenAI

print("✅ Import setup complete")
print(f"   Foundry Agents available: {foundry_agents_available}")

## Step 3.3: Initialize Azure AI Foundry Project Client

Set up the Azure AI Foundry project client for agent management.

In [ ]:
# Initialize Azure AI Foundry Project Client
project_client = None
azure_client = None

if foundry_agents_available and AZURE_AI_FOUNDRY_PROJECT_ENDPOINT:
    try:
        # Try to initialize with project connection string
        credential = DefaultAzureCredential()
        project_client = AIProjectClient(
            endpoint=AZURE_AI_FOUNDRY_PROJECT_ENDPOINT,
            credential=credential,  # Use Azure Default Credential for
                                                # authentication
        )
        print("✅ Azure AI Foundry Project Client initialized")
    except Exception as e:
        print(f"⚠️ Failed to initialize Project Client: {e}")
        print("   Will use direct Azure OpenAI client instead")
        project_client = None

# Initialize direct Azure OpenAI client as fallback
try:
    azure_client = AzureOpenAI(
        api_key=AZURE_OPENAI_KEY,
        api_version=AZURE_OPENAI_API_VERSION,
        azure_endpoint=AZURE_OPENAI_ENDPOINT
    )
    print("✅ Azure OpenAI client initialized (fallback)")
except Exception as e:
    print(f"❌ Failed to initialize Azure OpenAI client: {e}")

# Determine which approach to use
use_foundry_agents = project_client is not None
use_direct_openai = azure_client is not None

print(f"\n🎯 Available Approaches:")
print(f"   Azure AI Foundry Agents: {'✅ Available' if use_foundry_agents else '❌ Not available'}")
print(f"   Direct Azure OpenAI: {'✅ Available' if use_direct_openai else '❌ Not available'}")

if use_foundry_agents:
    print("\n🚀 Will use Azure AI Foundry Agents for advanced capabilities")
elif use_direct_openai:
    print("\n🔄 Will use direct Azure OpenAI client")
else:
    print("\n❌ No Azure services available. Please check configuration.")

## Step 3.4: Create Azure AI Foundry Agent

If Azure AI Foundry Agents are available, we'll create a specialized agent for our tasks.

In [ ]:
# Create Azure AI Foundry Agent (if available)
foundry_agent = None
agent_thread = None

if use_foundry_agents:
    try:
        # Create an agent with specific instructions and capabilities
        agent_instructions = """
        You are an intelligent AI assistant specialized in hybrid AI system analysis and documentation.
        Your primary capabilities include:
        1. Document analysis and summarization
        2. Technical explanation and education
        3. Business analysis and strategic recommendations
        4. Creative content generation
        5. Complex reasoning and problem-solving
        
        When responding:
        - Provide clear, well-structured answers
        - Include relevant examples when helpful
        - Acknowledge the complexity level of requests
        - Suggest when a task might be better suited for local vs cloud processing
        
        You are designed to handle sophisticated queries that require deep analysis,
        creativity, or extensive reasoning capabilities.
        """
        
        # Create agent
        foundry_agent = project_client.agents.create_agent(
            model=AZURE_OPENAI_DEPLOYMENT,
            name="Hybrid-AI-Specialist",
            instructions=agent_instructions.strip(),
            description="Specialized agent for hybrid AI system analysis and complex tasks"
        )
        
        print(f"✅ Azure AI Foundry Agent created:")
        print(f"   Agent ID: {foundry_agent.id}")
        print(f"   Agent Name: {foundry_agent.name}")
        print(f"   Model: {foundry_agent.model}")
        
        # Create a conversation thread
        agent_thread = project_client.agents.threads.create()
        print(f"   Thread ID: {agent_thread.id}")
        
    except Exception as e:
        print(f"❌ Failed to create Foundry Agent: {e}")
        print("   Will use direct Azure OpenAI approach")
        use_foundry_agents = False
        foundry_agent = None
        agent_thread = None

else:
    print("🔄 Using direct Azure OpenAI client (Foundry Agents not available)")

print(f"\n📊 Agent Setup Status:")
print(f"   Foundry Agent: {'✅ Ready' if foundry_agent else '❌ Not available'}")
print(f"   Direct Client: {'✅ Ready' if azure_client else '❌ Not available'}")

## Step 3.5: Create Unified Query Functions

Let's create functions that can use either Azure AI Foundry Agents or direct Azure OpenAI, depending on availability.

In [ ]:
def query_with_foundry_agent(prompt: str, thread_id: str = None) -> tuple:
    """Query using Azure AI Foundry Agent."""
    if not foundry_agent or not project_client:
        return "Foundry Agent not available", 0, False
    
    try:
        start_time = time.time()
        
        # Use existing thread or create new one
        if thread_id is None:
            current_thread = agent_thread
        else:
            current_thread = project_client.agents.threads.get(thread_id)
        
        # Add message to thread
        message = project_client.agents.messages.create(
            thread_id=current_thread.id,
            role="user",
            content=prompt
        )

        print(f"Created message, ID: {message.id}")
        
        # Create and wait for run
        run = project_client.agents.runs.create_and_process(
            thread_id=current_thread.id,
            agent_id=foundry_agent.id
        )

        print(f"Run finished with status: {run.status}")

        # Check if the run failed
        if run.status == "failed":
            print(f"Run failed: {run.last_error}")
            return f"Run failed: {run.last_error}", time.time() - start_time, False

        # Wait for completion (additional safety check)
        while run.status in ["in_progress", "queued"]:
            time.sleep(0.5)
            run = project_client.agents.runs.get(thread_id=current_thread.id, run_id=run.id)
        
        end_time = time.time()
        
        if run.status == "completed":
            # Get the latest message
            messages = project_client.agents.messages.list(thread_id=current_thread.id)
            
            # Convert ItemPaged to list and get the most recent message
            message_list = list(messages)
            if message_list:
                latest_message = message_list[0]  # Most recent message
                
                if latest_message.role == "assistant":
                    # Handle different content types
                    if hasattr(latest_message.content[0], 'text'):
                        content = latest_message.content[0].text.value
                    else:
                        content = str(latest_message.content[0])
                    return content, end_time - start_time, True
                else:
                    return "No assistant response found", end_time - start_time, False
            else:
                return "No messages found in thread", end_time - start_time, False
        else:
            return f"Run failed with status: {run.status}", end_time - start_time, False
            
    except Exception as e:
        return f"Foundry Agent error: {str(e)}", 0, False

def query_with_direct_openai(prompt: str, chat_history: list = None, max_tokens: int = 500) -> tuple:
    """Query using direct Azure OpenAI client."""
    if not azure_client:
        return "Azure OpenAI client not available", 0, False
    
    try:
        start_time = time.time()
        
        # Prepare messages
        if chat_history is None:
            messages = [{"role": "user", "content": prompt}]
        else:
            messages = chat_history + [{"role": "user", "content": prompt}]
        
        # Make API call
        response = azure_client.chat.completions.create(
            model=AZURE_OPENAI_DEPLOYMENT,
            messages=messages,
            max_tokens=max_tokens,
            temperature=0.7
        )
        
        end_time = time.time()
        content = response.choices[0].message.content
        return content, end_time - start_time, True
        
    except Exception as e:
        return f"Direct OpenAI error: {str(e)}", 0, False

def query_azure_intelligent(prompt: str, chat_history: list = None, max_tokens: int = 500) -> tuple:
    """Intelligently choose between Foundry Agent and direct OpenAI."""
    if use_foundry_agents:
        print("🤖 Using Azure AI Foundry Agent...")
        return query_with_foundry_agent(prompt)
    elif use_direct_openai:
        print("☁️ Using direct Azure OpenAI...")
        return query_with_direct_openai(prompt, chat_history, max_tokens)
    else:
        return "No Azure services available", 0, False

print("✅ Query functions created")
print(f"   Primary method: {'Foundry Agents' if use_foundry_agents else 'Direct OpenAI' if use_direct_openai else 'None'}")

## Step 3.6: Test Basic Azure Connectivity

Let's test our Azure connection with a simple query to ensure everything is working.

In [ ]:
# Test basic connectivity
test_prompt = "Hello! Please introduce yourself and describe your capabilities as an AI assistant."

print("🧪 Testing Azure connectivity and agent capabilities")
print("=" * 60)
print(f"Test query: {test_prompt}")
print()

response, response_time, success = query_azure_intelligent(test_prompt, max_tokens=200)

if success:
    print(f"✅ Connection successful!")
    print(f"⏱️ Response time: {response_time:.3f} seconds")
    print(f"📝 Response length: {len(response)} characters")
    print("\n🤖 Azure Response:")
    print("-" * 40)
    print(response)
    print("-" * 40)
else:
    print(f"❌ Connection failed: {response}")
    print("Please check your Azure configuration and try again.")

print("\n" + "=" * 60)

## Step 3.7: Test Complex Document Analysis

Now let's test the Azure system with complex document analysis tasks that showcase agent capabilities.

In [ ]:
# Sample business document for analysis
business_document = """
EXECUTIVE SUMMARY: HYBRID AI IMPLEMENTATION STRATEGY

Our organization is evaluating the implementation of a hybrid AI architecture that combines 
on-premises local models with cloud-based AI services. This strategic initiative aims to 
optimize performance, cost, and data privacy while maximizing AI capabilities.

KEY BENEFITS:
1. Performance Optimization: Local models provide sub-second response times for simple queries,
   while cloud models handle complex reasoning tasks.
2. Cost Management: Reduces cloud API costs by routing simple requests locally.
3. Data Privacy: Sensitive data can be processed locally without leaving the organization.
4. Scalability: Cloud resources handle peak loads and complex workloads.
5. Reliability: Local fallback ensures service continuity during network issues.

IMPLEMENTATION CHALLENGES:
- Model Management: Maintaining and updating local models requires expertise.
- Routing Logic: Intelligent decision-making for local vs cloud routing.
- Infrastructure: On-premises hardware requirements and maintenance.
- Integration: Seamless user experience across hybrid architecture.
- Monitoring: Comprehensive observability across local and cloud components.

FINANCIAL PROJECTIONS:
Initial investment: $150,000 (hardware, software, training)
Annual operational costs: $75,000
Projected savings: $200,000 annually in cloud API costs
ROI timeline: 12-18 months

RECOMMENDATIONS:
Proceed with pilot implementation focusing on customer service and document processing use cases.
Establish success metrics and evaluation criteria before full deployment.
Invest in team training and change management processes.
"""

# Test document analysis
analysis_prompt = f"""
Please analyze the following business document and provide:
1. A concise executive summary (2-3 sentences)
2. Key strategic recommendations
3. Potential risks and mitigation strategies
4. Assessment of the financial viability

Document:
{business_document}
"""

print("📊 Testing complex document analysis with Azure AI")
print("=" * 60)
print(f"Document length: {len(business_document)} characters")
print(f"Word count: ~{len(business_document.split())} words")
print("\nRequesting comprehensive analysis...\n")

analysis, response_time, success = query_azure_intelligent(analysis_prompt, max_tokens=600)

if success:
    print(f"✅ Analysis completed successfully")
    print(f"⏱️ Processing time: {response_time:.3f} seconds")
    print(f"📄 Analysis length: {len(analysis)} characters")
    print("\n📋 Business Document Analysis:")
    print("=" * 50)
    print(analysis)
    print("=" * 50)
    
    # Calculate analysis metrics
    compression_ratio = len(business_document) / len(analysis)
    words_per_second = len(analysis.split()) / response_time if response_time > 0 else 0
    
    print(f"\n📊 Analysis Metrics:")
    print(f"   Compression ratio: {compression_ratio:.1f}:1")
    print(f"   Processing speed: {words_per_second:.1f} words/second")
    print(f"   Analysis depth: {'Comprehensive' if len(analysis) > 800 else 'Moderate' if len(analysis) > 400 else 'Brief'}")
else:
    print(f"❌ Document analysis failed: {analysis}")

print("\n" + "=" * 60)

## Step 3.8: Test Creative and Strategic Tasks

Let's test the Azure system with tasks that require creativity and strategic thinking.

In [ ]:
# Creative and strategic tasks
advanced_tasks = [
    {
        "name": "Strategic Planning",
        "prompt": "Develop a comprehensive go-to-market strategy for a hybrid AI platform targeting enterprise customers. Include market positioning, pricing strategy, competitive analysis, and key success metrics.",
        "max_tokens": 500,
        "expected_elements": ["market", "pricing", "competitive", "metrics"]
    },
    {
        "name": "Technical Architecture",
        "prompt": "Design a scalable architecture for a hybrid AI system that can handle 10,000 concurrent users with both local and cloud processing. Include load balancing, failover mechanisms, and performance optimization strategies.",
        "max_tokens": 450,
        "expected_elements": ["scalable", "load", "failover", "optimization"]
    },
    {
        "name": "Risk Assessment",
        "prompt": "Conduct a comprehensive risk assessment for implementing hybrid AI in a financial services organization. Include regulatory, security, operational, and reputational risks with mitigation strategies.",
        "max_tokens": 400,
        "expected_elements": ["regulatory", "security", "operational", "mitigation"]
    }
]

print("🎯 Testing advanced strategic and creative capabilities")
print("=" * 70)

total_time = 0
successful_tasks = 0
task_results = []

for i, task in enumerate(advanced_tasks, 1):
    print(f"\n{i}. {task['name']}")
    print("-" * 50)
    print(f"📋 Task: {task['prompt'][:100]}...")
    print("\n🔄 Processing...")
    
    response, response_time, success = query_azure_intelligent(
        task['prompt'], 
        max_tokens=task['max_tokens']
    )
    
    if success:
        successful_tasks += 1
        total_time += response_time
        
        # Check for expected elements
        elements_found = sum(1 for element in task['expected_elements'] 
                           if element.lower() in response.lower())
        completeness = elements_found / len(task['expected_elements'])
        
        task_results.append({
            'name': task['name'],
            'response_time': response_time,
            'response_length': len(response),
            'completeness': completeness
        })
        
        print(f"\n✅ Task completed successfully")
        print(f"⏱️ Time: {response_time:.3f} seconds")
        print(f"📏 Length: {len(response)} characters")
        print(f"🎯 Completeness: {completeness:.1%} (found {elements_found}/{len(task['expected_elements'])} key elements)")
        print(f"\n🤖 Response Preview:")
        print(response[:300] + "..." if len(response) > 300 else response)
        
    else:
        print(f"❌ Task failed: {response}")
    
    print("\n" + "=" * 70)

# Summary statistics
if successful_tasks > 0:
    avg_response_time = total_time / successful_tasks
    avg_completeness = sum(r['completeness'] for r in task_results) / len(task_results)
    avg_length = sum(r['response_length'] for r in task_results) / len(task_results)
    
    print(f"\n📊 Advanced Task Performance Summary:")
    print(f"   Successful tasks: {successful_tasks}/{len(advanced_tasks)} ({successful_tasks/len(advanced_tasks):.1%})")
    print(f"   Average response time: {avg_response_time:.3f} seconds")
    print(f"   Average completeness: {avg_completeness:.1%}")
    print(f"   Average response length: {avg_length:.0f} characters")
    print(f"   Total processing time: {total_time:.3f} seconds")
    
    # Performance assessment
    if avg_response_time < 5 and avg_completeness > 0.7:
        print("\n🎉 Excellent performance - ready for production workloads!")
    elif avg_response_time < 10 and avg_completeness > 0.5:
        print("\n✅ Good performance - suitable for most enterprise use cases")
    else:
        print("\n⚠️ Performance may need optimization for production use")
else:
    print("\n❌ No tasks completed successfully. Please check configuration.")

## Step 3.9: Compare Agent vs Direct API Performance

If both methods are available, let's compare their performance characteristics.

In [ ]:
# Comparison test queries
comparison_queries = [
    "Explain the benefits of hybrid AI architectures in simple terms.",
    "What are the key considerations for implementing edge AI in retail?",
    "Summarize the main advantages of using local models vs cloud models."
]

print("⚖️ Comparing Azure AI Foundry Agent vs Direct OpenAI Performance")
print("=" * 70)

if use_foundry_agents and use_direct_openai:
    foundry_times = []
    direct_times = []
    
    for i, query in enumerate(comparison_queries, 1):
        print(f"\n{i}. Query: {query}")
        print("-" * 50)
        
        # Test with Foundry Agent
        print("🤖 Testing with Foundry Agent...")
        foundry_response, foundry_time, foundry_success = query_with_foundry_agent(query)
        
        if foundry_success:
            foundry_times.append(foundry_time)
            print(f"   ✅ Success ({foundry_time:.3f}s, {len(foundry_response)} chars)")
            print(f"   Preview: {foundry_response[:150]}...")
        else:
            print(f"   ❌ Failed: {foundry_response}")
        
        # Test with Direct OpenAI
        print("\n☁️ Testing with Direct OpenAI...")
        direct_response, direct_time, direct_success = query_with_direct_openai(query, max_tokens=300)
        
        if direct_success:
            direct_times.append(direct_time)
            print(f"   ✅ Success ({direct_time:.3f}s, {len(direct_response)} chars)")
            print(f"   Preview: {direct_response[:150]}...")
        else:
            print(f"   ❌ Failed: {direct_response}")
        
        # Compare if both succeeded
        if foundry_success and direct_success:
            speed_ratio = foundry_time / direct_time if direct_time > 0 else float('inf')
            print(f"\n📊 Comparison:")
            print(f"   Foundry Agent: {foundry_time:.3f}s")
            print(f"   Direct OpenAI: {direct_time:.3f}s")
            print(f"   Speed ratio: {speed_ratio:.2f}x {'(Agent slower)' if speed_ratio > 1 else '(Agent faster)'}")
        
        print("\n" + "="*50)
    
    # Overall comparison
    if foundry_times and direct_times:
        avg_foundry = sum(foundry_times) / len(foundry_times)
        avg_direct = sum(direct_times) / len(direct_times)
        
        print(f"\n🏆 Performance Summary:")
        print(f"   Foundry Agent average: {avg_foundry:.3f} seconds")
        print(f"   Direct OpenAI average: {avg_direct:.3f} seconds")
        
        if avg_foundry < avg_direct:
            improvement = ((avg_direct - avg_foundry) / avg_direct) * 100
            print(f"   🚀 Foundry Agent is {improvement:.1f}% faster")
        else:
            overhead = ((avg_foundry - avg_direct) / avg_direct) * 100
            print(f"   ⚠️ Foundry Agent has {overhead:.1f}% overhead")
            print(f"       (This overhead often provides additional capabilities)")

else:
    print("⚠️ Cannot compare - only one method available")
    if use_foundry_agents:
        print("   Available: Azure AI Foundry Agents")
    elif use_direct_openai:
        print("   Available: Direct Azure OpenAI")
    else:
        print("   Available: None")

print("\n" + "=" * 70)

## Step 3.10: Test Multi-turn Conversation Capabilities

Let's test the conversation management capabilities, especially if using Azure AI Foundry Agents.

In [ ]:
# Multi-turn conversation test
conversation_turns = [
    "I'm planning to implement a hybrid AI system for my company. Can you help me get started?",
    "What are the main technical considerations I should be aware of?",
    "How should I approach the cost-benefit analysis?",
    "Can you provide a rough timeline for implementation?",
    "What success metrics should I track?"
]

print("💬 Testing multi-turn conversation capabilities")
print("=" * 60)

if use_foundry_agents:
    print("🤖 Using Azure AI Foundry Agent (with thread management)")
    # Create a new thread for this conversation
    try:
        conversation_thread = project_client.agents.threads.create()
        print(f"   Created conversation thread: {conversation_thread.id}")
        
        for i, turn in enumerate(conversation_turns, 1):
            print(f"\n{i}. 👤 User: {turn}")
            
            response, response_time, success = query_with_foundry_agent(
                turn, thread_id=conversation_thread.id
            )
            
            if success:
                print(f"🤖 Agent ({response_time:.3f}s): {response[:200]}{'...' if len(response) > 200 else ''}")
            else:
                print(f"❌ Agent failed: {response}")
                break
        
        print(f"\n✅ Multi-turn conversation completed using thread {conversation_thread.id}")
        
    except Exception as e:
        print(f"❌ Thread management failed: {e}")
        print("   Falling back to direct OpenAI with manual history management")
        use_foundry_agents = False

if not use_foundry_agents and use_direct_openai:
    print("☁️ Using Direct Azure OpenAI (with manual history management)")
    
    chat_history = []
    
    for i, turn in enumerate(conversation_turns, 1):
        print(f"\n{i}. 👤 User: {turn}")
        
        response, response_time, success = query_with_direct_openai(
            turn, chat_history=chat_history, max_tokens=300
        )
        
        if success:
            print(f"🤖 Assistant ({response_time:.3f}s): {response[:200]}{'...' if len(response) > 200 else ''}")
            
            # Update chat history
            chat_history.append({"role": "user", "content": turn})
            chat_history.append({"role": "assistant", "content": response})
            
            # Keep history manageable (last 8 messages)
            if len(chat_history) > 8:
                chat_history = chat_history[-8:]
        else:
            print(f"❌ Assistant failed: {response}")
            break
    
    print(f"\n✅ Multi-turn conversation completed with {len(chat_history)} messages in history")

if not use_foundry_agents and not use_direct_openai:
    print("❌ No Azure services available for conversation testing")

print("\n" + "=" * 60)

## Step 3.11: Performance Analysis and Recommendations

Let's analyze the performance characteristics and provide recommendations for the hybrid system.

In [ ]:
# Performance analysis and recommendations
print("📊 Azure AI Foundry vs Direct OpenAI Analysis")
print("=" * 60)

# Foundry Agents analysis
if use_foundry_agents:
    print("\n🤖 Azure AI Foundry Agents - Strengths:")
    foundry_strengths = [
        "✅ Built-in conversation state management",
        "✅ Advanced thread and message handling",
        "✅ Native tool integration capabilities", 
        "✅ Structured agent instructions and behavior",
        "✅ Multi-turn conversation context preservation",
        "✅ Integrated with Azure AI services ecosystem",
        "✅ Automatic retry and error handling"
    ]
    
    for strength in foundry_strengths:
        print(f"   {strength}")
    
    print("\n🤔 Azure AI Foundry Agents - Considerations:")
    foundry_considerations = [
        "⚠️ Slightly higher latency due to agent orchestration",
        "⚠️ More complex setup and configuration",
        "⚠️ Additional dependency on Azure AI Projects service",
        "⚠️ May have usage limits different from direct API"
    ]
    
    for consideration in foundry_considerations:
        print(f"   {consideration}")

# Direct OpenAI analysis
if use_direct_openai:
    print("\n☁️ Direct Azure OpenAI - Strengths:")
    direct_strengths = [
        "✅ Lower latency for simple queries",
        "✅ Direct control over all parameters",
        "✅ Simpler implementation and debugging",
        "✅ Maximum compatibility with OpenAI APIs",
        "✅ Lower complexity and fewer dependencies",
        "✅ More predictable performance characteristics"
    ]
    
    for strength in direct_strengths:
        print(f"   {strength}")
    
    print("\n🤔 Direct Azure OpenAI - Considerations:")
    direct_considerations = [
        "⚠️ Manual conversation state management required",
        "⚠️ Custom implementation needed for advanced features",
        "⚠️ More code required for error handling and retries",
        "⚠️ Tool integration requires custom development"
    ]
    
    for consideration in direct_considerations:
        print(f"   {consideration}")

# Hybrid architecture recommendations
print("\n🎯 Hybrid Architecture Recommendations:")
print("-" * 40)

recommendations = {
    "Use Azure AI Foundry Agents for:": [
        "📝 Complex multi-turn conversations",
        "🔧 Applications requiring tool integration",
        "📊 Advanced workflow orchestration",
        "🎯 Applications needing sophisticated agent behavior",
        "📈 Enterprise scenarios with complex requirements"
    ],
    "Use Direct Azure OpenAI for:": [
        "⚡ Simple, fast query-response scenarios",
        "🔄 High-throughput applications",
        "🎛️ Applications needing fine-grained control",
        "🏗️ Custom integration scenarios",
        "💰 Cost-sensitive implementations"
    ]
}

for category, items in recommendations.items():
    print(f"\n{category}")
    for item in items:
        print(f"   {item}")

# Routing strategy for hybrid system
print("\n🧭 Routing Strategy for Hybrid System:")
print("-" * 40)

routing_strategy = {
    "Route to Local Models:": [
        "🏠 Simple queries and quick responses",
        "⚡ Latency-sensitive applications", 
        "🔒 Privacy-sensitive data processing",
        "💰 Cost optimization for high-volume queries"
    ],
    "Route to Azure AI Foundry Agents:": [
        "🤖 Complex reasoning and analysis tasks",
        "💬 Multi-turn conversations requiring context",
        "🔧 Tasks requiring tool integration",
        "📊 Strategic planning and business analysis"
    ],
    "Route to Direct Azure OpenAI:": [
        "⚡ Fast complex queries without conversation state",
        "📝 Document processing and summarization",
        "🎨 Creative content generation",
        "🔄 Batch processing scenarios"
    ]
}

for category, items in routing_strategy.items():
    print(f"\n{category}")
    for item in items:
        print(f"   {item}")

print("\n" + "=" * 60)
print("🎉 Azure AI analysis complete - ready for Lab 4 routing implementation!")

## Step 3.12: Create Helper Functions for Future Labs

Let's create reusable functions for both Azure AI Foundry Agents and direct OpenAI that we'll use in subsequent labs.

In [ ]:
# Create helper functions for future labs

class AzureAIManager:
    """Unified manager for Azure AI services (both Foundry Agents and direct OpenAI)."""
    
    def __init__(self):
        self.project_client = project_client
        self.azure_client = azure_client
        self.foundry_agent = foundry_agent
        self.agent_thread = agent_thread
        self.use_foundry_agents = use_foundry_agents
        self.use_direct_openai = use_direct_openai
        
    def query(self, prompt: str, method: str = "auto", **kwargs) -> tuple:
        """Query Azure AI with automatic or specified method selection."""
        if method == "auto":
            return query_azure_intelligent(prompt, **kwargs)
        elif method == "foundry" and self.use_foundry_agents:
            return query_with_foundry_agent(prompt, **kwargs)
        elif method == "direct" and self.use_direct_openai:
            return query_with_direct_openai(prompt, **kwargs)
        else:
            return f"Method '{method}' not available", 0, False
    
    def get_capabilities(self) -> dict:
        """Get information about available Azure AI capabilities."""
        return {
            "foundry_agents_available": self.use_foundry_agents,
            "direct_openai_available": self.use_direct_openai,
            "primary_method": "foundry" if self.use_foundry_agents else "direct" if self.use_direct_openai else "none",
            "deployment": AZURE_OPENAI_DEPLOYMENT,
            "endpoint": AZURE_OPENAI_ENDPOINT
        }
    
    def create_conversation_thread(self):
        """Create a new conversation thread (Foundry Agents only)."""
        if self.use_foundry_agents and self.project_client:
            try:
                return self.project_client.agents.threads.create()
            except Exception as e:
                print(f"Failed to create thread: {e}")
                return None
        return None

# Initialize the manager
azure_ai_manager = AzureAIManager()

# Test the manager
capabilities = azure_ai_manager.get_capabilities()
print("🔧 Azure AI Manager initialized")
print("📊 Capabilities:")
for key, value in capabilities.items():
    print(f"   {key}: {value}")

# Quick test
test_response, test_time, test_success = azure_ai_manager.query(
    "Briefly describe the benefits of using Azure AI Foundry for enterprise applications.",
    method="auto"
)

if test_success:
    print(f"\n✅ Manager test successful ({test_time:.3f}s)")
    print(f"📝 Response preview: {test_response[:150]}...")
else:
    print(f"\n❌ Manager test failed: {test_response}")

print("\n✅ Helper functions created and tested for future labs")
print("🔄 Azure AI Manager ready for integration with hybrid routing system")

## 🎉 Lab 3 (Alternate) Complete!

### What You've Accomplished:
- ✅ Successfully explored Azure AI Foundry Agents capabilities
- ✅ Implemented both agent-based and direct OpenAI approaches
- ✅ Tested complex document analysis and strategic planning tasks
- ✅ Compared performance characteristics between different methods
- ✅ Evaluated multi-turn conversation management
- ✅ Created a unified Azure AI manager for future integration

### Key Findings:

**Azure AI Foundry Agents:**
- **Strengths**: Advanced conversation management, built-in tool integration, sophisticated agent behavior
- **Best for**: Complex multi-turn scenarios, enterprise applications requiring advanced capabilities
- **Trade-offs**: Slightly higher latency, more complex setup

**Direct Azure OpenAI:**
- **Strengths**: Lower latency, simpler implementation, direct control
- **Best for**: Fast responses, high-throughput scenarios, simple integrations
- **Trade-offs**: Manual state management, custom feature development required

### Hybrid Architecture Strategy:

1. **Local Models**: Fast, private processing for simple queries
2. **Azure AI Foundry Agents**: Complex reasoning, multi-turn conversations, advanced workflows
3. **Direct Azure OpenAI**: Fast cloud processing, document analysis, creative tasks

### Next Steps:
- Proceed to Lab 4 to implement intelligent routing between local models and Azure AI services
- The Azure AI Manager is ready for integration with the hybrid routing system
- Performance characteristics will inform routing decisions

### Advanced Capabilities Unlocked:
✨ **Conversation State Management**: Automatic context preservation across interactions

🔧 **Tool Integration Ready**: Foundation for adding external tool capabilities

📊 **Enterprise-Grade**: Suitable for complex business applications

🎯 **Flexible Routing**: Multiple Azure AI approaches for different use cases

**Ready to build the intelligent three-way router in Lab 4!** 🚀

*This alternate Lab 3 provides a more sophisticated foundation for enterprise hybrid AI systems, leveraging the full capabilities of Azure AI Foundry while maintaining compatibility with direct OpenAI approaches.*